In [8]:
import streamlit as st
import pandas as pd
from datetime import datetime,timedelta
import bs4
import requests
from pykrx import stock
import matplotlib.pyplot as plt
import numpy as np
from pymongo import MongoClient
from itertools import combinations


from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression # 앙상블 조합용
from sklearn.neighbors import KNeighborsClassifier # 앙상블 조합용
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import warnings

warnings.filterwarnings('ignore')
my_client = MongoClient('mongodb://18.181.49.139/:27017')
mydb = my_client['final_project']

In [9]:
class Prediction():
    def __init__(self,company): # input : classifier,features
        self.basic_numeric = pd.DataFrame(mydb[f'{company}_all'].find()).drop(['_id'], axis=1)
        self.numeric = self.basic_numeric.loc[:len(self.basic_numeric), ['vix', '코스피', '코스닥', '개인매수량',
                                                                         '기관합계매수량', '외국인합계매수량', 'score1', 'score2',
                                                                         'score3', '등락률']]
        self.numeric = self.numeric.astype('float64')
        self.pred_target = self.basic_numeric.iloc[-1, :]
        # df 모양
        label_list = list(self.numeric['등락률'])
        tp = []

        for i in label_list:
            if i > 0:
                tp.append(1)
            else:
                tp.append(0)
        self.label = pd.DataFrame(tp)
        self.company = company

    # bundle(dtc(),)
    def bundle(self, classifier, features, test_size):

        total_tbl_train = self.numeric.loc[:, features]
        total_tbl_label = self.label.iloc[:, 0]

        X_train, X_test, y_train, y_test = train_test_split(total_tbl_train
                                                            , total_tbl_label
                                                            , test_size=test_size,
                                                            random_state=10)

        #DecisionTree = DecisionTreeClassifier(random_state=10)
        #Logistic = LogisticRegression(random_state=10)
        #KNeighbors = KNeighborsClassifier(n_neighbors=4)
        #Voting = VotingClassifier(estimators=[('LR', Logistic), ('KN', KNeighbors)], voting='hard')
        #RandomForest = RandomForestClassifier(random_state=10)
        #GradientBoosting = GradientBoostingClassifier(random_state=10)
        #XGB = xgb.XGBClassifier(random_state=10)
        #choice = {'DecisionTree': self.dtc(), 'Logistic': Logistic, 'KNeighbors': KNeighbors,
        #          'Voting': Voting, 'RandomForest': RandomForest, 'GradientBoosting': GradientBoosting,
        #          'XGB': XGB}

        clf = classifier
        clf.fit(X_train, y_train)  # classifier
        acc_pred = clf.predict(X_test)
        acc = np.round(accuracy_score(y_test, acc_pred), 4)

        pred = clf.predict(np.array(self.pred_target[features]).reshape(1,-1))
        if int(pred) == 1:
            st.success(f'다음날 {self.company} 주가는 상승할것으로 예측됩니다. 예측 정확도는 {round(acc * 100, 2)}% 입니다.')
        elif int(pred) == 0:
            st.success(f'다음날 {self.company} 주가는 하락할것으로 예측됩니다. 예측 정확도는 {round(acc * 100, 2)}% 입니다.')
        return round(acc * 100, 2),pred

    def dtc(self,max_depth,min_samples_leaf,min_samples_split):

        return DecisionTreeClassifier(random_state=10,
                                      max_depth=max_depth,
                                      min_samples_leaf=min_samples_leaf,
                                      min_samples_split=min_samples_split)

    def logi(self,l1_ratio):
        return LogisticRegression(random_state=10,penalty='elasticnet',l1_ratio=l1_ratio,solver='saga')

    def kNN(self,n_neighbors):
        return KNeighborsClassifier(n_neighbors=n_neighbors)

    def Vote(self):
        return VotingClassifier(estimators=[('LR', self.logi(0.5)), ('KN', self.kNN(4))])

    def RanF(self,max_depth,min_samples_leaf,min_samples_split,max_samples):
        return RandomForestClassifier(random_state=10,
                                      max_depth=max_depth,
                                      min_samples_leaf=min_samples_leaf,
                                      min_samples_split=min_samples_split,
                                      bootstrap=True,
                                      max_samples=max_samples)

    def GradB(self,max_depth,min_samples_leaf,min_samples_split,learning_rate):
        return GradientBoostingClassifier(random_state=10,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          min_samples_split=min_samples_split,
                                          learning_rate=learning_rate)

    def xgB(self):
        return xgb.XGBClassifier(random_state=10)


In [10]:
tp = Prediction('kakao')
tp.numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   vix       380 non-null    float64
 1   코스피       380 non-null    float64
 2   코스닥       380 non-null    float64
 3   개인매수량     380 non-null    float64
 4   기관합계매수량   380 non-null    float64
 5   외국인합계매수량  380 non-null    float64
 6   score1    380 non-null    float64
 7   score2    380 non-null    float64
 8   score3    380 non-null    float64
 9   등락률       380 non-null    float64
dtypes: float64(10)
memory usage: 29.8 KB


In [11]:


class best():
    def __init__(self,company):
        self.result = []
        print('start')
        self.result.append(self.dtc_best(company))
        print('dtc fin')
        self.result.append(self.logi_best(company))
        print('logi fin')
        self.result.append(self.knn_best(company))
        print('knn fin')
        self.result.append(self.vote_best(company))
        print('vote fin')
        self.result.append(self.ranf_best(company))
        print('ranf fin')
        self.result.append(self.grad_best(company))
        print('grad fin')
        self.result.append(self.xgb_best(company))
        print('xgb fin')


    def dtc_best(self,company):
        prediction_method = Prediction(company)
        train_test_rate = 0.2

        features = ['vix', '코스피', '코스닥', '개인매수량','기관합계매수량', '외국인합계매수량','score1', 'score2','score3']
        tp = []
        idx = []
        for i in range(1,10):
            for j in range(1,10):
                for k in range(2,10):
                    for l in range(1,len(features)+1):
                        for feature in list(combinations(features,l)):
                            np.random.seed(10)
                            #print(np.random.rand(5))
                            tp.append(prediction_method.bundle(prediction_method.dtc(i,j,k), list(feature), train_test_rate)[0])
                            idx.append([i,j,k,feature])
                            #print([i,j,k,feature])
        return 'dtc',idx[tp.index(max(tp))],max(tp)

    def logi_best(self,company):
        prediction_method = Prediction(company)
        train_test_rate = 0.2

        features = ['vix', '코스피', '코스닥', '개인매수량','기관합계매수량', '외국인합계매수량','score1', 'score2','score3']
        tp = []
        idx = []
        for i in range(0,101,5):
            for l in range(1,len(features)+1):
                for feature in list(combinations(features,l)):
                    np.random.seed(10)
                    #print(np.random.rand(5))
                    tp.append(prediction_method.bundle(prediction_method.logi(i/100), list(feature), train_test_rate)[0])
                    idx.append([i,feature])
                    #print([i/100,feature])

        return 'logi',idx[tp.index(max(tp))],max(tp)

    def knn_best(self,company):
        prediction_method = Prediction(company)
        train_test_rate = 0.2

        features = ['vix', '코스피', '코스닥', '개인매수량','기관합계매수량', '외국인합계매수량','score1', 'score2','score3']
        tp = []
        idx = []
        for i in range(1,11):
            for l in range(1,len(features)+1):
                for feature in list(combinations(features,l)):
                    np.random.seed(10)
                    #print(np.random.rand(5))
                    tp.append(prediction_method.bundle(prediction_method.kNN(i), list(feature), train_test_rate)[0])
                    idx.append([i,feature])
                    #print([i,feature])

        return 'knn',idx[tp.index(max(tp))],max(tp)

    def vote_best(self,company):
        prediction_method = Prediction(company)
        train_test_rate = 0.2

        features = ['vix', '코스피', '코스닥', '개인매수량','기관합계매수량', '외국인합계매수량','score1', 'score2','score3']
        tp = []
        idx = []

        for l in range(1,len(features)+1):
            for feature in list(combinations(features,l)):
                np.random.seed(10)
                #print(np.random.rand(5))
                tp.append(prediction_method.bundle(prediction_method.Vote(), list(feature), train_test_rate)[0])
                idx.append([feature])
                #print([feature])

        return 'vote',idx[tp.index(max(tp))],max(tp)

    def ranf_best(self,company):
        prediction_method = Prediction(company)
        train_test_rate = 0.2

        features = ['vix', '코스피', '코스닥', '개인매수량','기관합계매수량', '외국인합계매수량','score1', 'score2','score3']
        tp = []
        idx = []
        for i in range(1,10):
            for j in range(1,10):
                for k in range(2,10):
                    for ms in range(10,101,10):
                        #for l in range(1,len(features)+1):
                        #    for feature in list(combinations(features,l)):
                        np.random.seed(10)
                        #print(np.random.rand(5))
                        tp.append(prediction_method.bundle(prediction_method.RanF(i,j,k,ms), features, train_test_rate)[0])
                        idx.append([i,j,k,ms,features])
                        print([i,j,k,ms,features])
            print(f'{i}/10')

        return 'ranf',idx[tp.index(max(tp))],max(tp)

    def grad_best(self,company):
        prediction_method = Prediction(company)
        train_test_rate = 0.2

        features = ['vix', '코스피', '코스닥', '개인매수량','기관합계매수량', '외국인합계매수량','score1', 'score2','score3']
        tp = []
        idx = []
        for i in range(1,10):
            for j in range(1,10):
                for k in range(2,10):
                    for ms in range(1,5):
                        #for l in range(1,len(features)+1):
                        #    for feature in list(combinations(features,l)):
                        np.random.seed(10)
                        #print(np.random.rand(5))
                        tp.append(prediction_method.bundle(prediction_method.GradB(i,j,k,ms/10), list(features), train_test_rate)[0])
                        idx.append([i,j,k,ms/10,features])
                        print([i,j,k,ms/10,features])
            print(f'{i}/10')

        return 'grad',idx[tp.index(max(tp))],max(tp)

    def xgb_best(self,company):
        prediction_method = Prediction(company)
        train_test_rate = 0.2

        features = ['vix', '코스피', '코스닥', '개인매수량','기관합계매수량', '외국인합계매수량','score1', 'score2','score3']
        tp = []
        idx = []

        for l in range(1,len(features)+1):
            for feature in list(combinations(features,l)):
                np.random.seed(10)
                #print(np.random.rand(5))
                tp.append(prediction_method.bundle(prediction_method.xgB(), list(feature), train_test_rate)[0])
                idx.append([feature])
                #print([feature])

        return 'xgb',idx[tp.index(max(tp))],max(tp)

In [12]:
answer_n = best('naver')
#answer_n.result

answer_k = best('kakao')
#answer_k.result

start
dtc fin
logi fin
knn fin
vote fin
[1, 1, 2, 10, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 20, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 30, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 40, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 50, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 60, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 70, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 80, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 90, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 1, 2, 100, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 

[1, 2, 2, 40, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 2, 50, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 2, 60, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 2, 70, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 2, 80, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 2, 90, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 2, 100, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 3, 10, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 3, 20, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 3, 30, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 2, 3,

[1, 3, 2, 80, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 2, 90, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 2, 100, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 3, 10, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 3, 20, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 3, 30, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 3, 40, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 3, 50, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 3, 60, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 3, 70, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 3, 3,

[1, 4, 3, 20, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 3, 30, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 3, 40, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 3, 50, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 3, 60, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 3, 70, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 3, 80, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 3, 90, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 3, 100, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 4, 10, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 4, 4,

[1, 5, 3, 60, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 3, 70, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 3, 80, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 3, 90, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 3, 100, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 4, 10, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 4, 20, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 4, 30, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 4, 40, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 4, 50, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[1, 5, 4,

...

[8, 8, 7, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 7, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 7, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 8, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 8, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 8, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 8, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 9, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 9, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[8, 8, 9, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]


[9, 2, 3, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 4, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 4, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 4, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 4, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 5, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 5, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 5, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 5, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 2, 6, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]


[9, 4, 8, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 4, 8, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 4, 8, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 4, 9, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 4, 9, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 4, 9, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 4, 9, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 5, 2, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 5, 2, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 5, 2, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]


[9, 7, 4, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 5, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 5, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 5, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 5, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 6, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 6, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 6, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 6, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 7, 7, 0.1, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]


[9, 9, 9, 0.2, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 9, 9, 0.3, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
[9, 9, 9, 0.4, ['vix', '코스피', '코스닥', '개인매수량', '기관합계매수량', '외국인합계매수량', 'score1', 'score2', 'score3']]
9/10
grad fin
xgb fin


In [13]:
answer_n.result


[('dtc', [7, 2, 2, ('vix', '개인매수량', 'score3')], 90.79),
 ('logi', [0, ('개인매수량',)], 88.16),
 ('knn', [2, ('개인매수량',)], 88.16),
 ('vote', [('개인매수량',)], 86.84),
 ('ranf',
  [1,
   1,
   2,
   80,
   ['vix',
    '코스피',
    '코스닥',
    '개인매수량',
    '기관합계매수량',
    '외국인합계매수량',
    'score1',
    'score2',
    'score3']],
  89.47),
 ('grad',
  [1,
   1,
   2,
   0.1,
   ['vix',
    '코스피',
    '코스닥',
    '개인매수량',
    '기관합계매수량',
    '외국인합계매수량',
    'score1',
    'score2',
    'score3']],
  88.16),
 ('xgb', [('코스피', '개인매수량', 'score2')], 89.47)]

In [14]:
answer_k.result

[('dtc', [6, 1, 8, ('vix', '코스피', '개인매수량', '기관합계매수량', 'score3')], 93.42),
 ('logi', [0, ('개인매수량', '외국인합계매수량')], 92.11),
 ('knn', [8, ('개인매수량', '기관합계매수량')], 94.74),
 ('vote', [('개인매수량',)], 90.79),
 ('ranf',
  [4,
   1,
   2,
   80,
   ['vix',
    '코스피',
    '코스닥',
    '개인매수량',
    '기관합계매수량',
    '외국인합계매수량',
    'score1',
    'score2',
    'score3']],
  93.42),
 ('grad',
  [1,
   1,
   2,
   0.2,
   ['vix',
    '코스피',
    '코스닥',
    '개인매수량',
    '기관합계매수량',
    '외국인합계매수량',
    'score1',
    'score2',
    'score3']],
  94.74),
 ('xgb', [('코스닥', '개인매수량', 'score2')], 93.42)]